In [1]:
import pandas as pd
import pickle
import numpy as np
import time 
from datetime import datetime

time: 161 ms (started: 2022-06-07 08:50:07 +00:00)


# Data overview

## observe dataset date range

In [2]:
test_leader_df = pd.read_csv('../../recsys2022/test_leaderboard_sessions.csv')
test_df = pd.read_csv('../../recsys2022/test_final_sessions.csv')
test_leader_df['date'] =pd.to_datetime(test_leader_df['date'])
test_df['date'] =pd.to_datetime(test_df['date'])
print(test_leader_df['date'].max(), test_leader_df['date'].min(), test_df['date'].max(), test_df['date'].min())

2021-06-30 23:51:32.466000 2021-06-01 00:01:01.019000 2021-06-30 23:32:44.280000 2021-06-01 00:00:09.399000
time: 286 ms (started: 2022-06-07 08:50:54 +00:00)


## Observe  itemset  difference between dataset

In [ ]:
trn_df = pd.read_csv('../../recsys2022/train_sessions.csv')
trn_label_df = pd.read_csv('../../recsys2022/train_purchases.csv')
test_df = pd.read_csv('../../recsys2022/test_leaderboard_sessions.csv')
can_df = pd.read_csv('../../recsys2022/candidate_items.csv')
print(f"len of test item: {len(set(test_df['item_id']))}, item diff between test & train len: {len(set(test_df['item_id']) - set(trn_df['item_id']))}")
print(f"len of test item: {len(set(test_df['item_id']))}, item diff between candidate & test len: {len(set(can_df['item_id']) - set(test_df['item_id']))}")
print(f"len of train item: {len(set(trn_df['item_id']))}, item diff between candidate & trn len: {len(set(can_df['item_id']) - set(trn_df['item_id']))}")
print(f"len of train_label item: {len(set(trn_label_df['item_id']))}, item diff between candidate & trn len: {len(set(can_df['item_id']) - set(trn_label_df['item_id']))}")

In [ ]:
tmp = pd.concat([trn_df, trn_label_df, test_df])
print(trn_df.shape, trn_label_df.shape, test_df.shape, tmp.shape)
display(tmp)
print(set(can_df['item_id']) - set(tmp['item_id']))

# Data preprocessing

## concat test_leader & test_final 

In [9]:
test_leader_df = pd.read_csv('../../recsys2022/test_leaderboard_sessions.csv')
test_df = pd.read_csv('../../recsys2022/test_final_sessions.csv')
tmp = pd.concat([test_leader_df, test_df])
print(test_leader_df.shape, test_df.shape, tmp.shape)
display(tmp)
tmp.to_csv("../../recsys2022/test_leaderboard_final_sessions.csv")

(229354, 3) (226138, 3) (455492, 3)


,session_id,item_id,date
0,26,19185,2021-06-16 09:53:54.158
1,200,17089,2021-06-25 12:23:40.811
2,200,17089,2021-06-25 12:24:36.631
3,200,8060,2021-06-25 12:24:41.677
4,200,4758,2021-06-25 12:24:50.692
...,...,...,...
226133,4439648,7154,2021-06-14 08:03:19.024
226134,4439675,23067,2021-06-01 12:21:07.959
226135,4439868,26085,2021-06-16 22:18:27.509
226136,4439966,19483,2021-06-06 20:05:06.457


time: 677 ms (started: 2022-06-07 09:03:11 +00:00)


## item=>vector & session=>vector

## item==>vetor
### 1. mapping

In [14]:
def map_feature_label(target_path,dict_csv_path):
    all_feature=pd.read_csv(dict_csv_path)
    feature_label_divtionary={k:v for k,v in zip(all_feature['feature_name'],all_feature['label'])}
    item_feature=pd.read_csv(target_path)
    item_feature['feature_category_id'] = item_feature['feature_category_id'].apply(str)
    item_feature['feature_value_id'] = item_feature['feature_value_id'].apply(str)
    item_feature['feature_name']=item_feature["feature_category_id"] + "_" + item_feature["feature_value_id"]
    item_feature['label']=[feature_label_divtionary[i] for i in item_feature['feature_name']]
    item_feature['label'].apply(int)
    return item_feature

time: 574 µs (started: 2022-06-07 17:07:37 +00:00)


In [15]:
item_feature=map_feature_label('../../recsys2022/item_features.csv','../datasets/all_feature_encoding.csv')
item_feature["frequence"]=np.ones((len(item_feature),1))
item_feature

,item_id,feature_category_id,feature_value_id,feature_name,label,frequence
0,2,56,365,56_365,637,1.0
1,2,62,801,62_801,720,1.0
2,2,68,351,68_351,796,1.0
3,2,33,802,33_802,378,1.0
4,2,72,75,72_75,878,1.0
...,...,...,...,...,...,...
471746,28143,68,351,68_351,796,1.0
471747,28143,55,390,55_390,578,1.0
471748,28143,11,109,11_109,131,1.0
471749,28143,73,91,73_91,902,1.0


time: 407 ms (started: 2022-06-07 17:07:37 +00:00)


In [16]:
#pivot table
item_feature_p = item_feature.pivot_table(index="item_id", columns="feature_name",values="frequence")
item_feature_p = (item_feature_p.fillna(0))
item_feature_p.head()

feature_name,10_147,10_159,10_184,10_217,10_22,10_287,10_361,10_407,10_464,10_561,...,7_837,7_838,8_157,8_323,8_420,8_636,9_175,9_178,9_190,9_369
item_id,,,,,,,,,,,,,,,,,,,,,
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


time: 380 ms (started: 2022-06-07 17:07:39 +00:00)


## candidate_items => vector

In [17]:
def items2vec(_df_item,_session_items):
    session_vec = np.sum(_df_item.loc[pd.Index(_session_items)]).values
    return session_vec 

time: 325 µs (started: 2022-06-07 17:07:41 +00:00)


In [18]:
item_feature_reset_ind = item_feature_p.reset_index()
candidate_raw = pd.read_csv("../../recsys2022/candidate_items.csv")
candidate_vector = pd.merge(candidate_raw, item_feature_reset_ind, how='left', on="item_id")
candidate_vector.to_csv("../datasets/candidate_vector.csv", index=False)

time: 1.13 s (started: 2022-06-07 17:07:42 +00:00)


### retrieve the candidate only shows in specific period

In [21]:
purchases_last_df = pd.read_csv('../datasets/purchases_last_05m.csv')
purchases_last_itemset = set(purchases_last_df['item_id']) 
candidate_raw_itemset= set(candidate_raw['item_id']) 
candidate_last_itemset = purchases_last_itemset.intersection(candidate_raw_itemset)
candidate_last_df = pd.merge(candidate_raw, purchases_last_df['item_id']).drop_duplicates()
display(f"len of purchases_last_1_item_set:{len(purchases_last_itemset)}, len of candidate_raw_itemset:{len( candidate_raw_itemset)}, len of candidate_last_1_itemset :{len( candidate_last_itemset)}")
display(f"len of candidate_last_1_df :{len(candidate_last_df)}")

'len of purchases_last_1_item_set:4045, len of candidate_raw_itemset:4990, len of candidate_last_1_itemset :3742'

'len of candidate_last_1_df :3742'

time: 57.5 ms (started: 2022-06-07 17:09:25 +00:00)


In [ ]:
candidate_last_vector = pd.merge(candidate_last_df, item_feature_reset_ind, how='left', on="item_id")
candidate_last_vector.to_csv("../datasets/candidate_last_05m_vector.csv", index=False)

## Check length of  item set train&purchases

In [ ]:
purchases_raw_df = pd.read_csv("../../recsys2022/train_purchases.csv")
train_raw_df = pd.read_csv("../../recsys2022/train_sessions.csv")
purchases_itemset = set(purchases_raw_df['item_id'])
train_itemset = set(train_raw_df['item_id'])
display(purchases_raw_df.head(), train_raw_df.head())
display(f"len of purchase_item_set:{len(purchases_itemset)}, len of train_item_set:{len(train_itemset)}")